In [ ]:
import gradio as gr
import cv2
import json

# Your existing YOLOv8 + SORT pipeline
def process_video(video_path):
    # Process the video and generate violation clips and JSON data
    violation_clips = ["./violationClips/clip1.mp4", "./violationClips/clip2.mp4"]  # Example
    violation_data = [
        {"id": 1, "plate_number": "ABC123", "timestamp": "12:34:56"},
        {"id": 2, "plate_number": "XYZ789", "timestamp": "12:35:10"},
    ]
    return violation_clips, json.dumps(violation_data, indent=2)

# Gradio Interface
def gradio_interface(video):
    violation_clips, violation_data = process_video(video)
    return violation_clips, violation_data

# Define inputs and outputs
inputs = gr.Video(label="Upload Video")
outputs = [gr.Gallery(label="Violation Clips"), gr.JSON(label="Violation Data")]

# Launch the interface
interface = gr.Interface(
    fn=gradio_interface,
    inputs=inputs,
    outputs=outputs,
    title="Automatic Traffic Violation Detection",
    description="Upload a video to detect traffic violations.",
)

interface.launch()

In [5]:
import os
import cv2
import json
import numpy as np
import gradio as gr
from ultralytics import YOLO
from sort import *  # Make sure you have SORT installed

In [ ]:


# Initialize YOLOv8 model and SORT tracker
model = YOLO("yolov8n.pt")
tracker = Sort()

# Configuration
VIOLATION_CLIPS_DIR = "./violationClips"
os.makedirs(VIOLATION_CLIPS_DIR, exist_ok=True)

def detect_violations(video_path):
    """
    Process video to detect red light violations
    Returns: violation clips paths, violation data JSON
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")

    # Video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Violation tracking
    violations = []
    current_clip = None
    clip_writer = None
    violation_frames = []

    # Define your red light region (example coordinates)
    stop_line_y = int(height * 0.7)  # Adjust based on your video perspective
    red_light_active = False  # Replace with actual red light detection logic

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

        # Detect vehicles using YOLOv8
        results = model.track(frame, persist=True, classes=[2, 3, 5, 7])  # Car, bike, bus, truck
        boxes = results[0].boxes.xyxy.cpu().numpy()
        track_ids = results[0].boxes.id.cpu().numpy() if results[0].boxes.id is not None else []

        # Update SORT tracker
        tracked_objects = tracker.update(boxes)

        # Check for violations
        if red_light_active:
            for obj in tracked_objects:
                x1, y1, x2, y2, obj_id = map(int, obj)
                
                # Check if vehicle crosses stop line
                if y2 > stop_line_y:
                    # Start recording violation clip
                    if clip_writer is None:
                        clip_path = os.path.join(VIOLATION_CLIPS_DIR, f"violation_{obj_id}_{frame_number}.mp4")
                        clip_writer = cv2.VideoWriter(
                            clip_path,
                            cv2.VideoWriter_fourcc(*'mp4v'),
                            fps,
                            (width, height)
                        )
                        violation_frames = []

                    # Save violation data
                    violations.append({
                        "id": int(obj_id),
                        "timestamp": f"{frame_number//fps//60:02}:{frame_number//fps%60:02}",
                        "plate_number": "UNKNOWN",  # Add license plate recognition here
                        "clip_path": clip_path
                    })

        # Write frames to clip if recording
        if clip_writer is not None:
            clip_writer.write(frame)
            violation_frames.append(frame_number)
            
            # Stop recording after 5 seconds
            if len(violation_frames) > fps * 5:
                clip_writer.release()
                clip_writer = None

    cap.release()
    if clip_writer is not None:
        clip_writer.release()

    # Prepare output
    violation_clips = [v["clip_path"] for v in violations]
    return violation_clips, json.dumps(violations, indent=2)

# Gradio Interface
with gr.Blocks(title="Traffic Violation System") as demo:
    gr.Markdown("# 🚦 Automatic Traffic Violation Detection System")
    
    with gr.Row():
        with gr.Column():
            video_input = gr.Video(label="Upload Traffic Video", sources=["upload"])
            process_btn = gr.Button("Process Video")
        
        with gr.Column():
            gallery = gr.Gallery(label="Violation Clips", columns=3)
            json_output = gr.JSON(label="Violation Data")
    
    process_btn.click(
        fn=detect_violations,
        inputs=video_input,
        outputs=[gallery, json_output]
    )

if __name__ == "__main__":
    demo.launch(server_port=7861, share=True)

The final running code but need some modification by Maazi

In [1]:
import os
import cv2
import json
import time
import tempfile
import gradio as gr
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from ultralytics import YOLO
from sort import *
import pickle


In [ ]:


class TrafficViolationSystem:
    def __init__(self, config):
        self.config = config
        self.histograms = self.load_histograms()
        self.model = YOLO('yolov8n.pt')
        self.tracker = Sort(max_age=15, min_hits=3)
        self.violations = []
        self.red_light_intervals = []
        self.current_red_interval = None
        self.frame_buffer = []
        self.plate_cache = {}
        self.tracking_summary = defaultdict(dict)

    def load_histograms(self):
            histograms = {}
            for label, path in self.config['histogram_files'].items():
                with open(path, 'rb') as f:
                    histograms[label] = pickle.load(f)
            return histograms

    def calculate_histogram(self, frame):
        height, width = frame.shape[:2]
        x_center, y_center, w, h = self.config['traffic_light_bbox']
        x_min = int((x_center - w/2) * width)
        x_max = int((x_center + w/2) * width)
        y_min = int((y_center - h/2) * height)
        y_max = int((y_center + h/2) * height)
        
        roi = frame[y_min:y_max, x_min:x_max]
        hist = cv2.calcHist([roi], [0,1,2], None, [8,8,8], [0,256]*3)
        return cv2.normalize(hist, hist).flatten()

    def detect_traffic_light(self, frame):
        frame_hist = self.calculate_histogram(frame)
        best_match, best_score = None, -1
        for label, hist in self.histograms.items():
            score = cv2.compareHist(frame_hist, hist, cv2.HISTCMP_CORREL)
            if score > best_score:
                best_score, best_match = score, label
        return best_match, best_score

    def detect_vehicles(self, frame):
        results = self.model(frame)
        detections = []
        for box in results[0].boxes.data:
            x1, y1, x2, y2, conf, cls = map(float, box)
            if int(cls) == 2:  # Car class
                detections.append([x1, y1, x2, y2, conf])
        return np.array(detections) if detections else np.empty((0, 5))

    def is_within_violation_zone(self, x, y):
        x_center, y_center, w, h = self.config['violation_zone_bbox']
        x_min = int((x_center - w/2) * self.video_width)
        x_max = int((x_center + w/2) * self.video_width)
        y_min = int((y_center - h/2) * self.video_height)
        y_max = int((y_center + h/2) * self.video_height)
        return x_min <= x <= x_max and y_min <= y <= y_max

    def track_violations(self, frame, frame_number):
        detections = self.detect_vehicles(frame)
        tracked_objs = self.tracker.update(detections)
        
        for obj in tracked_objs:
            x1, y1, x2, y2, track_id = map(int, obj)
            cx, cy = (x1+x2)//2, (y1+y2)//2
            
            if not self.is_within_violation_zone(cx, cy):
                continue
            
            if track_id not in self.tracking_summary:
                self.tracking_summary[track_id] = {
                    'first_seen': frame_number,
                    'last_seen': frame_number,
                    'violation_frames': [],
                    'plate_number': 'UNKNOWN',
                    'bbox': (x1, y1, x2, y2)  # Store bounding box coordinates
                }
            else:
                # Update the bounding box to the last known position
                self.tracking_summary[track_id]['bbox'] = (x1, y1, x2, y2)
            
            self.tracking_summary[track_id]['last_seen'] = frame_number
            self.tracking_summary[track_id]['violation_frames'].append(frame_number)
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)

    def process_video(self):
        cap = cv2.VideoCapture(self.config['input_video'])
        self.video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        writer = cv2.VideoWriter(self.config['output_video'], 
                            cv2.VideoWriter_fourcc(*'mp4v'), 
                            fps, (self.video_width, self.video_height))
        
        red_active = False
        for frame_num in tqdm(range(total_frames), desc="Processing Frames"):
            ret, frame = cap.read()
            if not ret: break
            
            light_status, _ = self.detect_traffic_light(frame)
            
            # Red light state management
            if light_status == 'Red' and not red_active:
                red_active = True
                self.current_red_interval = {
                    'start_frame': frame_num,
                    'start_time': frame_num/fps,
                    'end_frame': None,
                    'end_time': None
                }
                self.frame_buffer = []
            elif light_status != 'Red' and red_active:
                red_active = False
                self.current_red_interval['end_frame'] = frame_num
                self.current_red_interval['end_time'] = frame_num/fps
                self.red_light_intervals.append(self.current_red_interval)
                self.save_redlight_clip()
                self.frame_buffer = []
            
            if red_active:
                self.track_violations(frame, frame_num)
                self.frame_buffer.append(frame.copy())
            
            writer.write(frame)
        
        cap.release()
        writer.release()
        self.process_license_plates()
        self.generate_output()

    def save_redlight_clip(self):
        if not self.frame_buffer:
            return
        
        clip_path = os.path.join(
            self.config['output_folder'],
            f"redlight_{len(self.red_light_intervals)}.mp4"
        )
        
        h, w = self.frame_buffer[0].shape[:2]
        writer = cv2.VideoWriter(clip_path, cv2.VideoWriter_fourcc(*'mp4v'), 
                            self.config['output_fps'], (w, h))
        for f in self.frame_buffer:
            writer.write(f)
        writer.release()

    def process_license_plates(self):
        api_key = self.config['plate_api_key']
        for track_id, data in tqdm(self.tracking_summary.items(), desc="Processing Plates"):
            try:
                if not data['violation_frames']:
                    continue
                    
                frame_num = data['violation_frames'][0]
                cap = cv2.VideoCapture(self.config['input_video'])
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
                ret, frame = cap.read()
                
                if ret and 'bbox' in data:
                    x1, y1, x2, y2 = data['bbox']
                    # Add safety checks for bounding box coordinates
                    x1 = max(0, x1)
                    y1 = max(0, y1)
                    x2 = min(frame.shape[1], x2)
                    y2 = min(frame.shape[0], y2)
                    
                    if x2 > x1 and y2 > y1:
                        cropped = frame[y1:y2, x1:x2]
                        cv2.imwrite('temp.jpg', cropped)
                        
                        plate = self.recognize_plate('temp.jpg', api_key)
                        data['plate_number'] = plate
                    else:
                        print(f"Invalid bbox for track {track_id}: {data['bbox']}")
            except Exception as e:
                print(f"Error processing track {track_id}: {str(e)}")
            finally:
                cap.release()

    def recognize_plate(self, image_path, api_key, retries=3):
        for _ in range(retries):
            try:
                with open(image_path, 'rb') as f:
                    response = requests.post(
                        'https://api.platerecognizer.com/v1/plate-reader/',
                        files={'upload': f},
                        headers={'Authorization': f'Token {api_key}'}
                    )
                if response.status_code == 200:
                    results = response.json().get('results', [])
                    return results[0]['plate'] if results else 'UNKNOWN'
            except Exception as e:
                time.sleep(2)
        return 'UNKNOWN'

    def generate_output(self):
        output_data = {
            'redlight_intervals': self.red_light_intervals,
            'violations': []
        }
        
        for track_id, data in self.tracking_summary.items():
            output_data['violations'].append({
                'track_id': track_id,
                'plate_number': data['plate_number'],
                'first_seen': data['first_seen'],
                'last_seen': data['last_seen'],
                'duration': (data['last_seen'] - data['first_seen'])/self.config['output_fps']
            })
        
        with open(self.config['json_output'], 'w') as f:
            json.dump(output_data, f, indent=2)

def create_gradio_interface():
    def process_video_wrapper(video_file):
        # Create temporary working directory
        # This is necessary because the TrafficViolationSystem 
        # class expects a folder to store violation clips
        
        
        config = {
            "input_video": video_file,
            "output_video": "output.mp4",
            "output_folder": "violationClips",
            "json_output": "violations.json",
            "output_fps": 25,
            "plate_api_key": "df9ea40d34e80d6149f57d04d8b1328d61920916",
            "histogram_files": {
                "Green": "./Histogram_Data/averaged_hist_Green.pkl",
                "Red": "./Histogram_Data/averaged_hist_Red.pkl",
                "Orange": "./Histogram_Data/averaged_hist_Orange.pkl",
                "None": "./Histogram_Data/averaged_hist_None.pkl"
            },
            "traffic_light_bbox": (0.485879, 0.577487, 0.016630, 0.051067),
            "violation_zone_bbox": (0.901061, 0.730515, 0.197877, 0.199434)
        }

        # Initialize and process
        system = TrafficViolationSystem(config)
        system.process_video()

        # Collect results
        violation_clips = [os.path.join(config['output_folder'], f) 
                         for f in os.listdir(config['output_folder'])]
        with open(config['json_output']) as f:
            violation_data = json.load(f)

        return violation_clips, config['output_video'], violation_data

    with gr.Blocks(title="Traffic Violation Detection System", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🚥 Automatic Traffic Violation Detection System")
        
        with gr.Row():
            with gr.Column(scale=2):
                video_input = gr.Video(label="Upload Traffic Video", sources=["upload"])
                process_btn = gr.Button("Analyze Video", variant="primary")
                
            with gr.Column(scale=3):
                with gr.Tabs():
                    with gr.TabItem("Violation Clips"):
                        clip_gallery = gr.Gallery(label="Detected Violations", columns=3)
                    with gr.TabItem("Processed Video"):
                        output_video = gr.Video(label="Annotated Video")
                    with gr.TabItem("Violation Data"):
                        json_output = gr.JSON(label="Violation Details")
        
        process_btn.click(
            fn=process_video_wrapper,
            inputs=video_input,
            outputs=[clip_gallery, output_video, json_output]
        )

    return demo

if __name__ == "__main__":
    demo = create_gradio_interface()
    demo.launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=False,
        show_error=True
    )

The code by Saim for clipping violations

In [1]:
import cv2
import numpy as np
import os
import pickle
import gradio as gr
import tempfile
import json
from ultralytics import YOLO
from collections import defaultdict
from tqdm import tqdm

class TrafficViolationSystem:
    def __init__(self, config):
        self.config = config
        self.histograms = self.load_histograms()
        self.model = YOLO('yolov8n.pt')
        self.violations = []
        self.frame_buffer = []
        self.tracking_summary = defaultdict(dict)
        self.video_width = None
        self.video_height = None

    def load_histograms(self):
        histograms = {}
        for label, path in self.config['histogram_files'].items():
            with open(path, 'rb') as f:
                histograms[label] = pickle.load(f)
        return histograms

    def calculate_histogram(self, frame):
        height, width = frame.shape[:2]
        x_center, y_center, w, h = self.config['traffic_light_bbox']
        x_min = int((x_center - w/2) * width)
        x_max = int((x_center + w/2) * width)
        y_min = int((y_center - h/2) * height)
        y_max = int((y_center + h/2) * height)
        
        roi = frame[y_min:y_max, x_min:x_max]
        hist = cv2.calcHist([roi], [0,1,2], None, [8,8,8], [0,256]*3)
        return cv2.normalize(hist, hist).flatten()

    def detect_traffic_light(self, frame):
        frame_hist = self.calculate_histogram(frame)
        best_match, best_score = None, -1
        for label, hist in self.histograms.items():
            score = cv2.compareHist(frame_hist, hist, cv2.HISTCMP_CORREL)
            if score > best_score:
                best_score, best_match = score, label
        return best_match, best_score

    def is_within_violation_zone(self, x, y):
        x_center, y_center, w, h = self.config['violation_zone_bbox']
        x_min = int((x_center - w/2) * self.video_width)
        x_max = int((x_center + w/2) * self.video_width)
        y_min = int((y_center - h/2) * self.video_height)
        y_max = int((y_center + h/2) * self.video_height)
        return x_min <= x <= x_max and y_min <= y <= y_max

    def draw_arrow(self, frame, x1, y1, x2, y2):
        center_x = (x1 + x2) // 2
        top_y = y1 - 150
        arrow_width = 50
        arrow_height = 100
        shaft_width = 20

        arrow_tip = (center_x, y1)
        left_corner = (center_x - arrow_width, top_y + arrow_height)
        right_corner = (center_x + arrow_width, top_y + arrow_height)

        shaft_top_left = (center_x - shaft_width, top_y)
        shaft_top_right = (center_x + shaft_width, top_y)
        shaft_bottom_left = (center_x - shaft_width, top_y + arrow_height)
        shaft_bottom_right = (center_x + shaft_width, top_y + arrow_height)

        arrow_head = np.array([arrow_tip, left_corner, right_corner], np.int32)
        arrow_shaft = np.array([shaft_top_left, shaft_bottom_left, shaft_bottom_right, shaft_top_right], np.int32)

        color = (0, 0, 255)
        cv2.fillPoly(frame, [arrow_head], color)
        cv2.fillPoly(frame, [arrow_shaft], color)

    def process_video(self):
        cap = cv2.VideoCapture(self.config['input_video'])
        self.video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        writer = cv2.VideoWriter(self.config['output_video'], 
                               cv2.VideoWriter_fourcc(*'mp4v'), 
                               fps, (self.video_width, self.video_height))
        
        red_active = False
        for frame_num in tqdm(range(total_frames), desc="Processing Frames"):
            ret, frame = cap.read()
            if not ret: break
            
            light_status, _ = self.detect_traffic_light(frame)
            
            if light_status == 'Red' and not red_active:
                red_active = True
                self.frame_buffer = []
            elif light_status != 'Red' and red_active:
                red_active = False
                self.save_redlight_clip()
                self.frame_buffer = []
            
            if red_active:
                results = self.model.track(frame, persist=True, classes=[2,3,5,7])
                boxes = results[0].boxes.xyxy.cpu().numpy()
                track_ids = results[0].boxes.id.cpu().numpy() if results[0].boxes.id is not None else []

                for box, track_id in zip(boxes, track_ids):
                    x1, y1, x2, y2 = map(int, box[:4])
                    cx, cy = (x1+x2)//2, (y1+y2)//2
                    
                    if self.is_within_violation_zone(cx, cy):
                        if track_id not in self.tracking_summary:
                            self.tracking_summary[track_id] = {
                                'first_seen': frame_num,
                                'last_seen': frame_num,
                                'violation_frames': [],
                                'plate_number': 'UNKNOWN',
                                'bbox': (x1, y1, x2, y2)
                            }
                        else:
                            self.tracking_summary[track_id]['bbox'] = (x1, y1, x2, y2)
                        
                        self.tracking_summary[track_id]['last_seen'] = frame_num
                        self.tracking_summary[track_id]['violation_frames'].append(frame_num)
                        self.draw_arrow(frame, x1, y1, x2, y2)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 2)
                        cv2.putText(frame, f"ID: {track_id}", (x1, y1-10), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)

                self.frame_buffer.append(frame.copy())
            
            writer.write(frame)
        
        cap.release()
        writer.release()
        self.generate_output()

    def save_redlight_clip(self):
        if not self.frame_buffer:
            return
        
        clip_path = os.path.join(
            self.config['output_folder'],
            f"redlight_{len(self.violations)}.mp4"
        )
        
        h, w = self.frame_buffer[0].shape[:2]
        writer = cv2.VideoWriter(clip_path, cv2.VideoWriter_fourcc(*'mp4v'), 
                               self.config['output_fps'], (w, h))
        for f in self.frame_buffer:
            writer.write(f)
        writer.release()
        self.violations.append(clip_path)

    def generate_output(self):
        output_data = {
            'violations': []
        }
        
        for track_id, data in self.tracking_summary.items():
            output_data['violations'].append({
                'track_id': int(track_id),
                'first_seen': data['first_seen'],
                'last_seen': data['last_seen'],
                'duration': (data['last_seen'] - data['first_seen'])/self.config['output_fps']
            })
        
        with open(self.config['json_output'], 'w') as f:
            json.dump(output_data, f, indent=2)

def create_gradio_interface():
    def process_video_wrapper(video_file, red_hist, green_hist, orange_hist):
        config = {
                "input_video": video_file,
                "output_video": "output.mp4",
                "output_folder": "violationClips",
                "json_output": "violations.json",
                "output_fps": 25,
                "histogram_files": {
                    "Green": "./Histogram_Data/averaged_hist_Green.pkl",
                    "Red": "./Histogram_Data/averaged_hist_Red.pkl",
                    "Orange": "./Histogram_Data/averaged_hist_Orange.pkl",
                    "None": "./Histogram_Data/averaged_hist_None.pkl"
                },
                "traffic_light_bbox": (0.485879, 0.577487, 0.016630, 0.051067),
                "violation_zone_bbox": (0.901061, 0.730515, 0.197877, 0.199434)
            }

        # Initialize and process
        system = TrafficViolationSystem(config)
        system.process_video()

        # Collect results
        violation_clips = [f for f in os.listdir(".") if f.endswith(".mp4") and "redlight" in f]
        violation_clips = [os.path.join(".", f) for f in violation_clips]

        with open(config['json_output']) as f:
            violation_data = json.load(f)

        return violation_clips, config['output_video'], violation_data

    with gr.Blocks(title="Traffic Violation Detection", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🚥 Automatic Traffic Violation Detection System")
        
        with gr.Row():
            with gr.Column(scale=2):
                video_input = gr.Video(label="Upload Traffic Video", sources=["upload"])
                process_btn = gr.Button("Analyze Video", variant="primary")
            
            with gr.Column(scale=3):
                with gr.Tabs():
                    with gr.TabItem("Violation Clips"):
                        clip_gallery = gr.Gallery(label="Detected Violations", columns=3)
                    with gr.TabItem("Processed Video"):
                        output_video = gr.Video(label="Annotated Video")
                    with gr.TabItem("Violation Data"):
                        json_output = gr.JSON(label="Violation Details")
        
        process_btn.click(
            fn=process_video_wrapper,
            inputs=video_input,
            outputs=[clip_gallery, output_video, json_output]
        )


    return demo

if __name__ == "__main__":
    demo = create_gradio_interface()
    demo.launch(
        server_name="0.0.0.0",
        server_port=7861,
        share=True
    )

/home/maazi/Desktop/Fyp/ProjectsTesting/Dashboard Testing/sort/myvenv/lib/python3.12/site-packages/gradio/utils.py:1017: UserWarning: Expected 4 arguments for function <function create_gradio_interface.<locals>.process_video_wrapper at 0x74d1e29d9da0>, received 1.
  warnings.warn(
/home/maazi/Desktop/Fyp/ProjectsTesting/Dashboard Testing/sort/myvenv/lib/python3.12/site-packages/gradio/utils.py:1021: UserWarning: Expected at least 4 arguments for function <function create_gradio_interface.<locals>.process_video_wrapper at 0x74d1e29d9da0>, received 1.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7861
* Running on public URL: https://cde2391eecf077b39a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/home/maazi/Desktop/Fyp/ProjectsTesting/Dashboard Testing/sort/myvenv/lib/python3.12/site-packages/gradio/helpers.py:968: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
Processing Frames:  30%|██▉       | 139/468 [00:03<00:09, 36.25it/s]


0: 384x640 8 cars, 5 motorcycles, 1 truck, 99.3ms
Speed: 4.1ms preprocess, 99.3ms inference, 10.4ms postprocess per image at shape (1, 3, 384, 640)


Processing Frames:  46%|████▌     | 216/468 [00:08<00:07, 34.38it/s]

...

Processing Frames: 100%|█████████▉| 466/468 [01:04<00:00,  4.48it/s]


0: 384x640 5 cars, 3 motorcycles, 1 truck, 95.4ms
Speed: 5.6ms preprocess, 95.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)


Processing Frames: 100%|█████████▉| 467/468 [01:04<00:00,  4.49it/s]


0: 384x640 5 cars, 2 motorcycles, 1 truck, 227.9ms
Speed: 24.8ms preprocess, 227.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


Processing Frames: 100%|██████████| 468/468 [01:04<00:00,  7.21it/s]
/home/maazi/Desktop/Fyp/ProjectsTesting/Dashboard Testing/sort/myvenv/lib/python3.12/site-packages/gradio/components/video.py:340: UserWarning: Video does not have browser-compatible container or codec. Converting to mp4.
  warnings.warn(
